<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Прогнозирование стоимости автомобиля по характеристикам

Задача: создать модель, которая будет предсказывать стоимость автомобиля по его характеристикам.

Оценка: оценка модели проводится по MAPE (Mean Absolute Percentage Error) по средней относительной ошибки прогноза.

По условию соревнования, нам нужно самостоятельно собрать обучающий датасет.

Поскольку спарсить данные с сайта avto.ru не получилось по объективным причинам (обновление сайта и недостаток опыта), модель будем строить на датасете из baseline.

In [1]:
import numpy as np 
import pandas as pd 
import pandas_profiling
import sys
from tqdm.notebook import tqdm
import category_encoders as ce
from catboost import CatBoostRegressor
#from lazypredict.Supervised import LazyRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, StackingRegressor


In [2]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

Python       : 3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 23:03:20) 
Numpy        : 1.18.5


In [3]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [4]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [5]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [6]:
pd.set_option('display.max_columns', None) # чтобы выводить все столбцы
pd.options.display.max_colwidth = 100   # задать кол-во символов для показа

# Setup

In [7]:
VERSION    = 16
DIR_TRAIN  = '../input/parsing-all-moscow-auto-ru-09-09-2020/' # подключил к ноутбуку внешний датасет
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

# Data

In [8]:
!ls '../input'

parsing-all-moscow-auto-ru-09-09-2020  sf-dst-car-price-prediction


In [9]:
train = pd.read_csv(DIR_TRAIN+'all_auto_ru_09_09_2020.csv') # датасет для обучения модели
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89378 entries, 0 to 89377
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              89377 non-null  object 
 1   brand                 89378 non-null  object 
 2   color                 89378 non-null  object 
 3   fuelType              89378 non-null  object 
 4   modelDate             89377 non-null  float64
 5   name                  89377 non-null  object 
 6   numberOfDoors         89377 non-null  float64
 7   productionDate        89378 non-null  int64  
 8   vehicleConfiguration  89377 non-null  object 
 9   vehicleTransmission   89377 non-null  object 
 10  engineDisplacement    89377 non-null  object 
 11  enginePower           89377 non-null  float64
 12  description           86124 non-null  object 
 13  mileage               89378 non-null  int64  
 14  Комплектация          89378 non-null  object 
 15  Привод             

In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              34686 non-null  object
 1   brand                 34686 non-null  object
 2   car_url               34686 non-null  object
 3   color                 34686 non-null  object
 4   complectation_dict    6418 non-null   object
 5   description           34686 non-null  object
 6   engineDisplacement    34686 non-null  object
 7   enginePower           34686 non-null  object
 8   equipment_dict        24690 non-null  object
 9   fuelType              34686 non-null  object
 10  image                 34686 non-null  object
 11  mileage               34686 non-null  int64 
 12  modelDate             34686 non-null  int64 
 13  model_info            34686 non-null  object
 14  model_name            34686 non-null  object
 15  name                  34686 non-null

In [12]:
train.sample(1)

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,price,start_date,hidden,model
27517,Внедорожник 5 дв.,HYUNDAI,97948F,бензин,2010.0,2.0 AT (150 л.с.) 4WD,5.0,2011,ALLROAD_5_DOORS AUTOMATIC 2.0,AUTOMATIC,2.0,150.0,"Машина 2011 года, куплена в 2012 году в ТЦ Кунцево, обслуживалась там же. Мною была куплена в тр...",74800,{'id': '0'},полный,LEFT,NaN,2.0,ORIGINAL,True,NaN,750000.0,2019-04-16T16:41:28Z,NaN,IX35


In [13]:
test.sample(1)

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
14006,внедорожник 5 дв.,NISSAN,https://auto.ru/cars/used/sale/nissan/x_trail/1101115522-74bdd6db/,серый,NaN,Автомобиль приобретался мной у официального дилера в 2012 году. Я его первый и единственный води...,2.5 LTR,169 N12,"{""engine-proof"":true,""tinted-glass"":true,""leather-gear-stick"":true,""esp"":true,""feedback-alarm"":t...",бензин,https://autoru.naydex.net/G87gDZg56/28c7a8vqVE/xGtzQkxaH-LvfLMYcqnNjeKZWSgfkOxAh1jWB0acClC2yBpE7...,133200,2010,"{""code"":""X_TRAIL"",""name"":""X-Trail"",""ru_name"":""X-трейл"",""morphology"":{},""nameplate"":{""code"":"""",""n...",X_TRAIL,2.5 CVT (169 л.с.) 4WD,5,1603280589,RUB,2012,1101115522,"{""id"":""7024211"",""displacement"":2488,""engine_type"":""GASOLINE"",""gear_type"":""ALL_WHEEL_DRIVE"",""tran...",ALLROAD_5_DOORS VARIATOR 2.5,вариатор,JAPANESE,1 владелец,8 лет и 4 месяца,Оригинал,полный,Левый,Не требует ремонта,Растаможен


Данные в трейне и тесте не одинаковы,некоторые переменные не совпадают либо имеют разный вид.  
Выделим совпадающие колонки и приведем их общему виду.

# Data Preprocessing

In [14]:
# Общие колонки
set(test.columns).intersection(train.columns)

{'bodyType',
 'brand',
 'color',
 'description',
 'engineDisplacement',
 'enginePower',
 'fuelType',
 'mileage',
 'modelDate',
 'name',
 'numberOfDoors',
 'productionDate',
 'vehicleConfiguration',
 'vehicleTransmission',
 'Владельцы',
 'Владение',
 'ПТС',
 'Привод',
 'Руль',
 'Состояние',
 'Таможня'}

Пройдемся по каждой отдельно и объединим в общий датасет

In [15]:
# Создадим пустой список для имен колонок
columns = []

## Кузов

In [16]:
train.bodyType.unique(), test.bodyType.unique()

(array(['Седан', 'Универсал 5 дв.', 'Хэтчбек 5 дв. Sportback',
        'Хэтчбек 3 дв.', 'Хэтчбек 5 дв.', 'Кабриолет', 'Купе',
        'Лифтбек Sportback', 'Лифтбек', 'Седан Long', 'Внедорожник 5 дв.',
        'Кабриолет Roadster', 'Седан 2 дв.', 'Седан Gran Coupe',
        'Компактвэн', 'Компактвэн Gran Tourer', 'Лифтбек Gran Turismo',
        'Хэтчбек 3 дв. Compact', 'Лифтбек Gran Coupe', 'Купе-хардтоп',
        'Родстер Roadster', 'Родстер', 'Внедорожник 5 дв. ESV', 'Минивэн',
        'Пикап Двойная кабина', 'Внедорожник 3 дв.',
        'Пикап Одинарная кабина', 'Тарга', 'Пикап Двойная кабина Crew Cab',
        'Пикап Двойная кабина Double',
        'Пикап Одинарная кабина Regular Cab', 'Внедорожник 5 дв. EXT',
        'Седан SRT8', 'Минивэн SWB', 'Минивэн Grand', 'Компактвэн Grand',
        'Универсал 5 дв. CrossTourer', 'Минивэн Long', 'Минивэн XL',
        'Микровэн Coach', 'Хэтчбек 5 дв. Best', 'Хэтчбек 5 дв. SRT4',
        'Купе SRT', 'Седан SRT', 'Пикап Полуторная кабина',
    

Очень много похожих вариантов типов кузова.  
Имеет смысл объединить их в более общие категории.

In [17]:
# Отфильтруем по первому слову
train.bodyType = train.bodyType.apply(lambda x: x.lower().split()[0].strip() if isinstance(x, str) else x)
test.bodyType = test.bodyType.apply(lambda x: x.lower().split()[0].strip() if isinstance(x, str) else x)

# Переименуем колонку
train.rename(columns={'bodyType': 'Кузов'}, inplace=True)
test.rename(columns={'bodyType': 'Кузов'}, inplace=True)

columns.append('Кузов')

## Бренд

In [18]:
train.brand.unique(), test.brand.unique()

(array(['AUDI', 'BMW', 'CADILLAC', 'CHERY', 'CHEVROLET', 'CHRYSLER',
        'CITROEN', 'DAEWOO', 'DODGE', 'FORD', 'GEELY', 'HONDA', 'HYUNDAI',
        'INFINITI', 'JAGUAR', 'JEEP', 'KIA', 'LEXUS', 'MAZDA', 'MINI',
        'MITSUBISHI', 'NISSAN', 'OPEL', 'PEUGEOT', 'PORSCHE', 'RENAULT',
        'SKODA', 'SUBARU', 'SUZUKI', 'TOYOTA', 'VOLKSWAGEN', 'VOLVO',
        'GREAT_WALL', 'LAND_ROVER', 'MERCEDES', 'SSANG_YONG'], dtype=object),
 array(['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
        'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI'],
       dtype=object))

В трейне представленно гораздо больше различных брендов автомобилей чем в тесте.  
Удалим из трейна лишние бренды

In [19]:
# Переименуем колонку
train.rename(columns={'brand': 'Бренд'}, inplace=True)
test.rename(columns={'brand': 'Бренд'}, inplace=True)

# Оставим только бренды из теста
train = train[train['Бренд'].isin(test['Бренд'].unique())]

columns.append('Бренд')

## Цвет

In [20]:
train.color.unique(), test.color.unique()

(array(['040001', 'EE1D19', '0000CC', 'CACECB', '007F00', 'FAFBFB',
        '97948F', '22A0F8', '660099', '200204', 'C49648', 'DEA522',
        '4A2197', 'FFD600', 'FF8649', 'FFC0CB'], dtype=object),
 array(['синий', 'чёрный', 'серый', 'коричневый', 'белый', 'пурпурный',
        'бежевый', 'серебристый', 'красный', 'зелёный', 'жёлтый',
        'голубой', 'оранжевый', 'фиолетовый', 'золотистый', 'розовый'],
       dtype=object))

Цвета совпадают но имеют разную кодировку.  
Перекодируем в более читаемый вид.

In [21]:
# Словарь перекодировки цветов
hex_to_rus = {'040001':'чёрный', 
 'FAFBFB':'белый', 
 '97948F':'серый', 
 'CACECB':'серебристый', 
 '0000CC':'синий', 
 '200204':'коричневый', 
 'EE1D19':'красный', 
 '007F00':'зелёный', 
 'C49648':'бежевый', 
 '22A0F8':'голубой', 
 '660099':'пурпурный', 
 'DEA522':'золотистый', 
 '4A2197':'фиолетовый', 
 'FFD600':'жёлтый', 
 'FF8649':'оранжевый', 
 'FFC0CB':'розовый', 
}

In [22]:
# Заменим названия цветов в трейне
train.color = train.color.replace(to_replace = hex_to_rus)

# Переименуем колонку
train.rename(columns={'color': 'Цвет'}, inplace=True)
test.rename(columns={'color': 'Цвет'}, inplace=True)

columns.append('Цвет')

## Топливо

In [23]:
train.fuelType.unique(), test.fuelType.unique()

(array(['бензин', 'дизель', 'электро', 'гибрид', 'универсал', 'газ'],
       dtype=object),
 array(['бензин', 'дизель', 'гибрид', 'электро', 'газ'], dtype=object))

Оставим переменную как есть

In [24]:
# Переименуем колонку
train.rename(columns={'fuelType': 'Топливо'}, inplace=True)
test.rename(columns={'fuelType': 'Топливо'}, inplace=True)

columns.append('Топливо')

## Год_модели

In [25]:
train.modelDate.unique(), test.modelDate.unique()

(array([1990., 1982., 1988., 1976., 1983., 1979., 1986., 1991., 1978.,
        1987., 1992., 1938., 2010., 2014., 2000., 2016., 2004., 2012.,
        2008., 1996., 2003., 2015., 2011., 2007., 1999., 1994., 2009.,
        2018., 2001., 1997., 2006., 2019., 2013., 2017., 2002., 2005.,
        1980., 1984., 1998., 1966., 1975., 1937., 1936., 1949., 2020.,
        1995., 1981., 1972., 1977., 1989., 1968., 1993., 1985.,   nan,
        1973., 1969., 1934., 1960., 1974., 1950., 1961., 1955., 1965.,
        1904., 1963., 1971., 1959., 1958., 1951., 1957.]),
 array([2013, 2017, 2008, 2009, 2016, 2012, 2015, 2010, 2006, 2000, 2007,
        1994, 2004, 1999, 2005, 1976, 2001, 1969, 1996, 1998, 1989, 1934,
        2014, 2011, 2018, 1986, 1997, 1990, 2019, 2002, 1991, 1987, 1980,
        1982, 1938, 1988, 2003, 1983, 1978, 1979, 1984, 1992, 1995, 1993,
        1985, 1974, 1966, 1977, 1981, 1972, 1968, 1975, 1949, 1937, 1936,
        1973, 1959, 1958, 2020, 1965, 1971, 1904, 1963, 1955, 1951, 1960])

В трейне есть пропуски. Их мало поэтому просто удалим.  
Изменим тип данных в трейне на integer.

In [26]:
# Удалим пропуски
train.dropna(subset=['modelDate'], inplace=True)

# Преобразуем в int
train.modelDate = train.modelDate.apply(lambda x: int(x))

# Переименуем колонку
train.rename(columns={'modelDate': 'Год_модели'}, inplace=True)
test.rename(columns={'modelDate': 'Год_модели'}, inplace=True)

columns.append('Год_модели')

## Количество_дверей

In [27]:
train.numberOfDoors.unique(), test.numberOfDoors.unique()

(array([4., 5., 3., 2., 0.]), array([5, 4, 2, 3, 0]))

В данных есть нулевое значение что достаточно не логично для данной переменной.  
Заменим 0 на наиболее часто значение.  
Изменим тип данных в трейне на integer.

In [28]:
# Заменим 0
train.numberOfDoors.replace(0, 5, inplace=True)
test.numberOfDoors.replace(0, 5, inplace=True)

# Преобразуем в int
train.numberOfDoors = train.numberOfDoors.apply(lambda x: int(x))

# Переименуем колонку
train.rename(columns={'numberOfDoors': 'Количество_дверей'}, inplace=True)
test.rename(columns={'numberOfDoors': 'Количество_дверей'}, inplace=True)

columns.append('Количество_дверей')

## Год_выпуска

In [29]:
train.productionDate.unique(), test.productionDate.unique()

(array([1991, 1986, 1989, 1993, 1992, 1994, 1987, 1988, 1985, 1983, 1980,
        1984, 1990, 1981, 1995, 1938, 2014, 2011, 2013, 2012, 2016, 2015,
        2010, 2002, 2000, 2001, 2020, 2008, 2018, 2019, 2007, 2003, 2017,
        2006, 2005, 2004, 2009, 1997, 1999, 1998, 1996, 1982, 1972, 1978,
        1937, 1949, 1948, 1950, 1953, 1975, 1969, 1979, 1976, 1939, 1974,
        1973, 1954, 1964, 1970, 1961, 1960, 1957, 1968, 1977, 1904, 1967,
        1966, 1965, 1963, 1959, 1936, 1952]),
 array([2014, 2017, 2012, 2011, 2019, 2018, 2010, 2020, 2016, 2013, 2006,
        2007, 2015, 2005, 2008, 2009, 1997, 2004, 2002, 1987, 2003, 2001,
        1976, 2000, 1998, 1995, 1999, 1993, 1939, 1996, 1984, 1990, 1991,
        1992, 1989, 1982, 1985, 1994, 1938, 1981, 1988, 1983, 1980, 1986,
        1978, 1970, 1979, 1977, 1972, 1975, 1969, 1950, 1953, 1949, 1937,
        1959, 1968, 1936, 1904, 1974, 1967, 1961, 1960, 1965, 1963, 1957,
        1952, 1973, 1948]))

Оставим переменную как есть

In [30]:
# Переименуем колонку
train.rename(columns={'productionDate': 'Год_выпуска'}, inplace=True)
test.rename(columns={'productionDate': 'Год_выпуска'}, inplace=True)

columns.append('Год_выпуска')

## КПП

In [31]:
train.vehicleTransmission.unique(), test.vehicleTransmission.unique()

(array(['MECHANICAL', 'AUTOMATIC', 'ROBOT', 'VARIATOR'], dtype=object),
 array(['роботизированная', 'механическая', 'автоматическая', 'вариатор'],
       dtype=object))

Значения совпадают но нужно привести к общему виду.

In [32]:
# Заменим названия в трейне
train.vehicleTransmission = train.vehicleTransmission.replace(to_replace = {'MECHANICAL':'механическая',
                                                                           'AUTOMATIC':'автоматическая',
                                                                           'ROBOT':'роботизированная',
                                                                           'VARIATOR':'вариатор'})

# Переименуем колонку
train.rename(columns={'vehicleTransmission': 'КПП'}, inplace=True)
test.rename(columns={'vehicleTransmission': 'КПП'}, inplace=True)

columns.append('КПП')

## Мощность_двигателя

In [33]:
train.enginePower.unique(), test.enginePower.unique()

(array([174.,  90., 136., 101., 133., 150., 115., 137., 112., 230.,  70.,
         88., 100.,  75., 165., 182., 170., 113.,  80.,  54.,  60.,  71.,
        122., 125.,  61., 102., 190., 180., 160., 105., 140., 110., 250.,
        200., 130., 120., 225., 163., 211., 249., 255., 143., 220., 239.,
        265., 116., 193., 177., 240., 245., 204., 340., 300., 233., 310.,
        218., 210., 290., 333., 335., 224., 350., 155., 254., 338., 435.,
        372., 460., 275., 500., 260., 450., 420., 280., 326., 285., 408.,
        271., 272., 270., 238., 252., 610., 525., 430., 367., 605., 560.,
        580., 256., 354., 571., 360., 520., 400., 156., 306., 320., 184.,
        129., 258., 231., 118., 192., 286.,  98., 234., 171.,  46.,  51.,
        407., 530., 313., 235., 381., 462., 197., 109., 609., 544., 445.,
        329., 188., 370., 410., 431., 343., 507., 600., 625., 510., 355.,
        269., 264., 347., 555., 575., 321., 201., 281., 131., 154., 152.,
        147., 135.,  53., 215.,  44., 

Оставим в тесте только численное значение.  
Изменим тип данных на integer.

In [34]:
# Преобразуем в int и почистим test
train.enginePower = train.enginePower.apply(lambda x: int(x))
test.enginePower = test.enginePower.apply(lambda x: int(x[:-4]))

# Переименуем колонку
train.rename(columns={'enginePower': 'Мощность_двигателя'}, inplace=True)
test.rename(columns={'enginePower': 'Мощность_двигателя'}, inplace=True)

columns.append('Мощность_двигателя')

## Пробег

In [35]:
train.mileage.unique(), test.mileage.unique()

(array([350000, 173424, 230000, ..., 526000,  58726,    520]),
 array([ 74000,  60563,  88000, ..., 121276, 212678, 157965]))

Пробовал прологарифмировать показатель, но это не принесло улчшения метрики.

In [36]:
# Прологарифмируем показатель
#train.mileage = np.log(data.mileage + 1)
#test.mileage = np.log(data.mileage + 1)

# Переименуем колонку
train.rename(columns={'mileage': 'Пробег'}, inplace=True)
test.rename(columns={'mileage': 'Пробег'}, inplace=True)

columns.append('Пробег')

## Привод

In [37]:
train.Привод.unique(), test.Привод.unique()

(array(['полный', 'передний', 'задний'], dtype=object),
 array(['передний', 'полный', 'задний'], dtype=object))

Оставим переменную как есть.

In [38]:
columns.append('Привод')

## Руль

In [39]:
train.Руль.unique(), test.Руль.unique()

(array(['LEFT', 'RIGHT'], dtype=object),
 array(['Левый', 'Правый'], dtype=object))

Значения совпадают но нужно привести к общему виду.

In [40]:
# Заменим названия в трейне
train.Руль = train.Руль.replace(to_replace = {'LEFT':'Левый', 'RIGHT':'Правый'})

columns.append('Руль')

## Владельцы

In [41]:
train.Владельцы.unique(), test.Владельцы.unique()

(array([ 3.,  1.,  2., nan]),
 array(['3 или более', '1\xa0владелец', '2\xa0владельца'], dtype=object))

В трейне есть пропуски, заменим их на медианное значение.  
Изменим тип данных на integer.

In [42]:
# Заменим пропуски
train.Владельцы = train.Владельцы.fillna(value = train.Владельцы.median())

# Преобразуем в int и почистим test
train.Владельцы = train.Владельцы.apply(lambda x: int(x))
test.Владельцы = test.Владельцы.apply(lambda x: int(x[0]))

columns.append('Владельцы')

## ПТС

In [43]:
train.ПТС.unique(), test.ПТС.unique()

(array(['ORIGINAL', 'DUPLICATE', nan], dtype=object),
 array(['Оригинал', 'Дубликат', nan], dtype=object))

Приведем значения к общему виду и заменим пропуски.

In [44]:
# Заменим названия в трейне
train.ПТС = train.ПТС.replace(to_replace = {'ORIGINAL':'Оригинал', 'DUPLICATE':'Дубликат'})

# Обработаем пропуски
train.ПТС = train.ПТС.fillna(value = 'Оригинал')
test.ПТС = test.ПТС.fillna(value = 'Оригинал')

columns.append('ПТС')

## Модель

In [45]:
train.model.unique(), test.model_name.unique()

(array(['100', '200', '80', '90', '920', 'A1', 'A2', 'A3', 'A4',
        'A4_ALLROAD', 'A5', 'A6', 'ALLROAD', 'A7', 'A8', 'COUPE', 'E_TRON',
        'Q3', 'Q3_SPORTBACK', 'Q5', 'Q7', 'Q8', 'R8', 'RS3', 'RS4', 'RS5',
        'RS6', 'RS7', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'SQ5', 'TT',
        'TT_RS', 'TTS', 'V8', '02', '1ER', 'M1', '2ER', '2ACTIVETOURER',
        '2GRANDTOURER', '3ER', '321', '326', '340', '4', '5ER', '6ER',
        '7ER', '8ER', 'E3', 'I3', 'I8', 'M2', 'M3', 'M4', 'M5', 'M6', 'M8',
        'X1', 'X2', 'X3', 'X3_M', 'X4', 'X5', 'X5_M', 'X6', 'X6_M', 'X7',
        'Z1', 'Z3', 'Z3M', 'Z4', 'ACCORD', 'ACTY', 'AIRWAVE', 'ASCOT',
        'AVANCIER', 'CITY', 'CIVIC', 'CIVIC_FERIO', 'CIVIC_TYPE_R',
        'CONCERTO', 'CR_V', 'CR_X', 'CR_Z', 'CROSSROAD', 'CROSSTOUR',
        'DOMANI', 'EDIX', 'ELEMENT', 'ELYSION', 'FIT', 'FR_V', 'FREED',
        'HR_V', 'INSIGHT', 'INSPIRE', 'INTEGRA', 'JAZZ', 'LEGEND', 'LIFE',
        'LOGO', 'MOBILIO', 'MOBILIO_SPIKE', 'N_BOX', 'N_ONE', '

Пока оставим без изменений.

In [46]:
# Переименуем колонку
train.rename(columns={'model': 'Модель'}, inplace=True)
test.rename(columns={'model_name': 'Модель'}, inplace=True)

columns.append('Модель')

## Цена

Дата парсинга тестовой выборки -- октябрь 20-го,  
что всего на месяц отличается от приложенной к бейзлайну демонстрационной выборке для обучения.  
Поэтому не корректируем цену автомобилей для этой выборки.

In [47]:
# удалим пропуски
train.dropna(subset=['price'], inplace=True)

In [48]:
pd.Timestamp(test.parsing_unixtime.mean(), unit='s')

Timestamp('2020-10-21 13:25:33.619356394')

# Pandas Profiling

In [49]:
df_train = train[columns]
df_test = test[columns]

In [50]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83786 entries, 0 to 83785
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Кузов               83786 non-null  object
 1   Бренд               83786 non-null  object
 2   Цвет                83786 non-null  object
 3   Топливо             83786 non-null  object
 4   Год_модели          83786 non-null  int64 
 5   Количество_дверей   83786 non-null  int64 
 6   Год_выпуска         83786 non-null  int64 
 7   КПП                 83786 non-null  object
 8   Мощность_двигателя  83786 non-null  int64 
 9   Пробег              83786 non-null  int64 
 10  Привод              83786 non-null  object
 11  Руль                83786 non-null  object
 12  Владельцы           83786 non-null  int64 
 13  ПТС                 83786 non-null  object
 14  Модель              83786 non-null  object
 15  sample              83786 non-null  int64 
dtypes: int64(7), object(9)

In [52]:
data

,Кузов,Бренд,Цвет,Топливо,Год_модели,Количество_дверей,Год_выпуска,КПП,Мощность_двигателя,Пробег,Привод,Руль,Владельцы,ПТС,Модель,sample
0,лифтбек,SKODA,синий,бензин,2013,5,2014,роботизированная,105,74000,передний,Левый,3,Оригинал,OCTAVIA,0
1,лифтбек,SKODA,чёрный,бензин,2017,5,2017,механическая,110,60563,передний,Левый,1,Оригинал,OCTAVIA,0
2,лифтбек,SKODA,серый,бензин,2013,5,2014,роботизированная,152,88000,передний,Левый,1,Оригинал,SUPERB,0
3,лифтбек,SKODA,коричневый,бензин,2013,5,2014,автоматическая,110,95000,передний,Левый,1,Оригинал,OCTAVIA,0
4,лифтбек,SKODA,белый,бензин,2008,5,2012,автоматическая,152,58536,передний,Левый,1,Оригинал,OCTAVIA,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83781,купе,MERCEDES,золотистый,бензин,1951,2,1952,механическая,150,45200,задний,Левый,1,Оригинал,W188,1
83782,седан,MERCEDES,чёрный,бензин,1957,4,1959,автоматическая,160,10000,задний,Левый,2,Оригинал,W189,1
83783,пикап,MERCEDES,серый,дизель,2017,4,2018,автоматическая,258,800,полный,Левый,1,Оригинал,X_KLASSE,1
83784,пикап,MERCEDES,серый,дизель,2017,4,2018,автоматическая,190,15850,полный,Левый,1,Оригинал,X_KLASSE,1


In [53]:
pandas_profiling.ProfileReport(data)

Выводы:
* Большиинство автомобилей с пробегом больше 100 тыс км.
* Год выпуска марки и год производства авто сильно коррелируют друг с другом, но удалять пока ничего не будем.
* В пробеге много нулевых значений, вероятно это новые автомобили
* Превалируют авто с количеством владельцев 3, с автоматической кобобкой передач и с передним либо полным приводом.
* 4 марки (MERCEDES ,BMW, VOLKSWAGEN, NISSAN) составляют больше 30 % от числа всех остальных брендов.
* Много видов кузова, превалирют "седан" и "внедорожник".
* Из 16 цветов самый распространенный черный, за ним идут белый, серебристый, серый и синий.
* Тип топлива превалирует -бензин.
* В осном все авто пяти и четырехдверные.

# Feature Engineering

In [54]:
# Создадим бинарный признак для автомобилей старше 10 лет
data['Старое_авто'] = data.Год_выпуска.apply(lambda x: 0 if (2020-x)<10 else 1)

In [55]:
# Создадим бинарный признак для автомобилей с пробегом больше 100000 км
data['Высокий_пробег'] = data.Пробег.apply(lambda x: 1 if x > 100000 else 0)

# Label Encoding

In [56]:
# отбираем и кодируем категориальные признаки
cat_columns = ['Кузов', 'Бренд', 'Цвет', 'Топливо', 'КПП', 'Привод', 'Руль', 'ПТС', 'Модель']

for colum in cat_columns:
    data[colum] = data[colum].astype('category').cat.codes

#data = ce.BackwardDifferenceEncoder(cols = cat_columns).fit_transform(data, verbose=1) #ухудшило метрику
#data = ce.PolynomialEncoder(cols = cat_columns).fit_transform(data, verbose=1) #ухудшило метрику
#data = ce.OneHotEncoder(cols = cat_columns).fit_transform(data, verbose=1) #ухудшило метрику

In [57]:
data

,Кузов,Бренд,Цвет,Топливо,Год_модели,Количество_дверей,Год_выпуска,КПП,Мощность_двигателя,Пробег,Привод,Руль,Владельцы,ПТС,Модель,sample,Старое_авто,Высокий_пробег
0,6,8,13,0,2013,5,2014,3,105,74000,1,0,3,1,329,0,0,0
1,6,8,15,0,2017,5,2017,2,110,60563,1,0,1,1,329,0,0,0
2,6,8,12,0,2013,5,2014,3,152,88000,1,0,1,1,475,0,0,0
3,6,8,6,0,2013,5,2014,0,110,95000,1,0,1,1,329,0,0,0
4,6,8,1,0,2008,5,2012,0,152,58536,1,0,1,1,329,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83781,3,5,5,0,1951,2,1952,2,150,45200,0,0,1,1,539,1,1,0
83782,11,5,15,0,1957,4,1959,0,160,10000,0,0,2,1,540,1,1,0
83783,9,5,12,3,2017,4,2018,0,258,800,2,0,1,1,562,1,0,0
83784,9,5,12,3,2017,4,2018,0,190,15850,2,0,1,1,562,1,0,0


# Train Split

In [58]:
y = train['price']

X = data.query('sample == 1').drop(['sample'], axis=1)
X_sub = data.query('sample == 0').drop(['sample'], axis=1)

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# Lazy Predict

In [60]:
#reg  = LazyRegressor(verbose=0,
#                     ignore_warnings=True,
#                     custom_metric=None)
#models, predictions = reg .fit(X_train, X_test, y_train, y_test)
#models

In [61]:
# пришлось брать ограниченную выборку и запускать на google collab
# топ 5 полученных моделей
lazy_models = [ 'ExtraTreesRegressor',
               'RandomForestRegressor', 
               'XGBRegressor',
               'KNeighborsRegressor', 
               'DecisionTreeRegressor']

# Model 1 : CatBoost
У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. https://catboost.ai
На данный момент CatBoost является одной из лучших библиотек для табличных данных!

### Base

In [62]:
#model = CatBoostRegressor(iterations = 5000,
#                          random_seed = RANDOM_SEED,
#                          eval_metric='MAPE',
#                          custom_metric=['R2', 'MAE'],
#                          silent=True,
#                         )
#model.fit(X_train, y_train,
#         #cat_features=cat_features_ids,
#         eval_set=(X_test, y_test),
#         verbose_eval=0,
#         use_best_model=True,
#         #plot=True
#         )
#
#model.save_model('catboost_single_model_baseline.model')

In [63]:
## оцениваем точность
#predict = model.predict(X_test)
#print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

Точность модели по метрике MAPE: 13.79%

In [64]:
#pd.Series(model.get_feature_importance(), index = X_train.columns).plot(kind='bar')   #.sort_values(ascending=False)

### Log Traget
Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    
В принциепе мы можем использовать любое приобразование на целевую переменную. Например деление на курс доллара, евро или гречки :) в дату сбора данных, смотрим дату парсинга в тесте в **parsing_unixtime**

In [65]:
#np.log(y_train)

In [66]:
#model = CatBoostRegressor(iterations = 5000,
#                          random_seed = RANDOM_SEED,
#                          eval_metric='MAPE',
#                          custom_metric=['R2', 'MAE'],
#                          silent=True,
#                         )
#model.fit(X_train, np.log(y_train),
#         #cat_features=cat_features_ids,
#         eval_set=(X_test, np.log(y_test)),
#         verbose_eval=0,
#         use_best_model=True,
#         #plot=True
#         )
#
#model.save_model('catboost_single_model_2_baseline.model')

In [67]:
#predict_test = np.exp(model.predict(X_test))
#predict_submission = np.exp(model.predict(X_sub))

In [68]:
#print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 11.89%

### Best params

In [69]:
#подберем оптимальные параметры для нашей модели
#model = CatBoostRegressor(iterations = 6000, random_seed = RANDOM_SEED, eval_metric='MAPE', silent=True)

#grid = {'learning_rate': [0.03, 0.1], 'depth': [4, 6, 10], 'l2_leaf_reg': [1, 3, 5, 7, 9]}

#randomized_search_result = model.randomized_search(grid, X_train, y_train, plot=True)

In [70]:
##полученные параметры
#cbr_best_params = {'iterations': 6000,
#                'loss_function': 'RMSE',
#                'random_seed': 42,
#                'silent': True,
#                'eval_metric': 'MAPE',
#                'depth': 10,
#                'l2_leaf_reg': 3,
#                'learning_rate': 0.03}

In [71]:
#model = CatBoostRegressor(**cbr_best_params)
#
#model.fit(X_train, np.log(y_train), 
#          eval_set=(X_test, np.log(y_test)), 
#          verbose_eval=0, 
#          use_best_model=True)
#
#model.save_model('catboost_single_model_2_baseline.model')

In [72]:
#predict_test = np.exp(model.predict(X_test)) 
#
#print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 11.83%

# Model 2: XGBoost

### Base

In [73]:
#xgb_reg = xgb.XGBRegressor(random_state=RANDOM_SEED)
#
#xgb_reg.fit(X_train, np.log(y_train))

In [74]:
#predict_test = np.exp(xgb_reg.predict(X_test))
#
#print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 15.12% без логарифмирования целевой переменной  
Точность модели по метрике MAPE: 12.82% c логарифмированием целевой переменной

### Best params

In [75]:
#xgb_reg = xgb.XGBRegressor(
#    objective='reg:squarederror', 
#    colsample_bytree=0.5,               
#    learning_rate=0.05, 
#    max_depth=12, 
#    alpha=1,                   
#    n_estimators=1000,
#    random_state=RANDOM_SEED,
#    verbose=1, 
#    n_jobs=-1)
#
#
#xgb_reg.fit(X_train, np.log(y_train))

In [76]:
#predict_test = np.exp(xgb_reg.predict(X_test))
#
#print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 11.81%

# Model 3: ExtraTreesRegressor

### Base

In [77]:
#et_reg = ExtraTreesRegressor(random_state=RANDOM_SEED)
#
#et_reg.fit(X_train,np.log(y_train))

In [78]:
#predict_test = np.exp(et_reg.predict(X_test))
#
#print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 14.30% без логарифмирования целевой переменной  
Точность модели по метрике MAPE: 12.75% c логарифмированием целевой переменной

### Best params

In [79]:
#random_grid = {'n_estimators': [int(x) for x in np.linspace(start=100, stop=1000, num=10)], 
#               'max_features': ['auto', 'sqrt', 'log2'], 
#               'max_depth': [int(x) for x in np.linspace(5, 15, num=6)] + [None], 
#               'min_samples_split': [2, 5, 10], 
#               'min_samples_leaf': [1, 2, 4], 
#               'bootstrap': [True, False]}
#
#et_reg = ExtraTreesRegressor(random_state=RANDOM_SEED) 
#etr_random = RandomizedSearchCV(estimator=et_reg, param_distributions=random_grid, n_iter=100, 
#                                cv=3, verbose=1, random_state=RANDOM_SEED, n_jobs=-1)
#
#etr_random.fit(X_train, np.log(y_train)) 
#etr_random.best_params_

In [80]:
#etr_best_params = {'n_estimators': 700, 
#                   'min_samples_split': 5, 
#                   'min_samples_leaf': 1, 
#                   'max_features': 'auto', 
#                   'max_depth': None, 
#                   'bootstrap': False}

In [81]:
#et_reg = ExtraTreesRegressor(random_state = RANDOM_SEED, 
#                             n_estimators = 700, 
#                             min_samples_split = 5, 
#                             min_samples_leaf = 1, 
#                             max_features = 'auto', 
#                             max_depth = None, 
#                             bootstrap = False)
#
#et_reg.fit(X_train,np.log(y_train))

In [82]:
#predict_test = np.exp(et_reg.predict(X_test))
#
#print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 12.39%

# Model 4: RandomForestRegressor

### Base

In [83]:
#rf_reg = RandomForestRegressor(random_state=RANDOM_SEED)
#
#rf_reg.fit(X_train,np.log(y_train))

In [84]:
#predict_test = np.exp(rf_reg.predict(X_test))
#
#print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 14.20% без логарифмирования целевой переменной  
Точность модели по метрике MAPE: 12.67% c логарифмированием целевой переменной

### Best params

In [85]:
##Подбор гирерпараметров для модели:
#random_grid = {'n_estimators': [int(x) for x in np.linspace(start=100, stop=1000, num=10)], 
#               'max_features': ['sqrt', 'log2'],
#               'max_depth': [int(x) for x in np.linspace(1, 10, num=10)] + [None],
#               'min_samples_split': [2, 5, 10],
#               'min_samples_leaf': [1, 2, 4],
#               'bootstrap': [True, False]}
#
#rf_reg = RandomForestRegressor(random_state=RANDOM_SEED)
#rf_random = RandomizedSearchCV(estimator=rf_reg, param_distributions=random_grid,
#                               n_iter=100, cv=3, verbose=1, random_state=RANDOM_SEED, n_jobs=-1)
#
#rf_random.fit(X_train, np.log(y_train))
#rf_random.best_params_

In [86]:
##полученные параметры
#rfr_best_params = {'n_estimators': 1000, 
#                   'min_samples_split': 5, 
#                   'min_samples_leaf': 1, 
#                   'max_features': 'sqrt', 
#                   'max_depth': None, 
#                   'bootstrap': False}

In [87]:
#rf_reg = RandomForestRegressor(random_state = RANDOM_SEED, 
#                               n_estimators = 1000, 
#                               min_samples_split = 5, 
#                               min_samples_leaf = 1, 
#                               max_features = 'sqrt', 
#                               max_depth = None, 
#                               bootstrap = False)
#
#rf_reg.fit(X_train,np.log(y_train))

In [88]:
#predict_test = np.exp(rf_reg.predict(X_test))
#
#print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 12.08%

# Model 5: StackingRegressor

In [89]:
estimators = [('cbr', CatBoostRegressor(iterations = 6000, loss_function = 'RMSE', random_seed = RANDOM_SEED, 
                                         silent = True, eval_metric = 'MAPE', depth = 10, 
                                         l2_leaf_reg = 3, learning_rate = 0.03)),
              ('xgb_reg',xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, 
                                          learning_rate=0.05, max_depth=12, alpha=1, n_estimators=1000, 
                                          random_state=RANDOM_SEED, verbose=1, n_jobs=-1)),
              ('etr', ExtraTreesRegressor(random_state = RANDOM_SEED, n_estimators = 700, 
                                          min_samples_split = 5, min_samples_leaf = 1, max_features = 'auto', 
                                          max_depth = None, bootstrap = False)), 
              ('rfr', RandomForestRegressor(random_state = RANDOM_SEED, n_estimators = 1000, 
                                            min_samples_split = 5, min_samples_leaf = 1, max_features = 'sqrt', 
                                            max_depth = None, bootstrap = False))
              ]

st_ensemble = StackingRegressor(estimators = estimators, final_estimator = LinearRegression())

st_ensemble.fit(X_train, np.log(y_train)) 

[11:27:06] WARNING: ../src/learner.cc:516: 
Parameters: { verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[11:40:29] WARNING: ../src/learner.cc:516: 
Parameters: { verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[11:41:15] WARNING: ../src/learner.cc:516: 
Parameters: { verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[11:42:01] WARNING:

StackingRegressor(estimators=[('cbr',
                               <catboost.core.CatBoostRegressor object at 0x7f13a9d12dd0>),
                              ('xgb_reg',
                               XGBRegressor(alpha=1, base_score=None,
                                            booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.5, gamma=None,
                                            gpu_id=None, importance_type='gain',
                                            interaction_constraints=None,
                                            learning_rate=0.05,
                                            max_delta_step=None, max_depth=12,
                                            mi...
                                            reg_lambda=None,
                                            scale_pos_weight=None,
           

In [90]:
predict_test = np.exp(st_ensemble.predict(X_test)) 

print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 11.60%


Точность модели по метрике MAPE: 11.63%  
Точность модели по метрике MAPE: 11.60% после Feature Engineering

# Submission

In [91]:
predict_submission = np.exp(st_ensemble.predict(X_sub))

In [92]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_2_v{VERSION}.csv', index=False)
sample_submission.head(10)

,sell_id,price
0,1100575026,6.033971e+05
1,1100549428,9.126500e+05
2,1100658222,8.943242e+05
3,1100937408,7.348247e+05
4,1101037972,7.518561e+05
5,1100912634,7.550344e+05
6,1101228730,5.616096e+05
7,1100165896,3.788883e+05
8,1100768262,1.950385e+06
9,1101218501,7.876761e+05


# What's next?
Или что еще можно сделать, чтоб улучшить результат:

* Спарсить свежие данные 
* Посмотреть, что еще можно извлечь из признаков или как еще можно их обработать
* Сгенерировать больше новых признаков
* Попробовать другие алгоритмы и библиотеки ML